>### Imports

In [2]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt

from tensorflow import keras
from tensorflow.keras import layers

AUTOTUNE = tf.data.AUTOTUNE

c:\Users\sel04327\AppData\Local\Programs\Python\Python37\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


>### Download the training dataset
- cache() 메서드는 데이터셋을 빠르게 액세스하기 위해 메모리나 디스크에 캐시하는 데 사용.
- 데이터셋을 캐시하면 훈련 프로세스의 성능을 크게 향상
    - 특히 데이터셋이 메모리에 맞지 않을 경우, 캐시를 사용하는 것이 유용합니다. 
    - 또한 I/O 대역폭이 제한되거나 데이터셋 전처리 시간이 오래 걸리는 경우에도 유용합니다.
- 데이터셋을 캐시함으로써 불필요한 데이터 로딩 및 전처리 작업을 피할 수 있으며, 이는 시간을 절약하고 I/O 작업을 saving
    - 이는 특히 대용량 데이터셋이나 다중 에포크를 요구하는 딥러닝 모델을 훈련할 때 유용합니다.

In [4]:
# Download DIV2K from TF Datasets
# Using bicubic 4x degradation type
div2k_data = tfds.image.Div2k(config='bicubic_x4')
div2k_data.download_and_prepare()

# Taking train data from div2k_data object
train = div2k_data.as_dataset(split='train', as_supervised=True)
train_cache = train.cache()
# validation data
val = div2k_data.as_dataset(split='validation', as_supervised=True)
val_cache = val.cache()

EXTRACTING {'train_lr_url': 'https://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_LR_bicubic_X4.zip', 'valid_lr_url': 'https://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_valid_LR_bicubic_X4.zip', 'train_hr_url': 'https://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_HR.zip', 'valid_hr_url': 'https://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_valid_HR.zip'}


Dl Completed...: 0 url [00:00, ? url/s]
Dl Completed...:  25%|██▌       | 1/4 [00:17<00:44, 14.82s/ url]

Dl Completed...:  25%|██▌       | 1/4 [00:17<00:44, 14.82s/ url]

Dl Completed...:  25%|██▌       | 1/4 [00:17<00:44, 14.82s/ url]

Dl Completed...:  25%|██▌       | 1/4 [00:17<00:44, 14.82s/ url]

Dl Completed...:  25%|██▌       | 1/4 [00:17<00:44, 14.82s/ url]

Dl Completed...:  25%|██▌       | 1/4 [00:17<00:44, 14.82s/ url]

Dl Completed...:  25%|██▌       | 1/4 [00:18<00:44, 14.82s/ url]

Dl Completed...:  25%|██▌       | 1/4 [00:18<00:44, 14.82s/ url]

Dl Completed...:  25%|██▌       | 1/4 [00:18<00:44, 14.82s/ url]

Dl Completed...:  25%|██▌       | 1/4 [00:30<00:44, 14.82s/ url]

Dl Completed...:  50%|█████     | 2/4 [01:21<01:19, 39.62s/ url]

Dl Completed...:  50%|█████     | 2/4 [01:22<01:19, 39.62s/ url]

Dl Completed...:  50%|█████     | 2/4 [01:22<01:19, 39.62s/ url]

Dl Completed...:  50%|█████     | 2/4 [01:22<01:19, 39.62s/ url]

Dl Completed...:  50%|█████     | 2/

Dataset div2k downloaded and prepared to C:\Users\sel04327\tensorflow_datasets\div2k\bicubic_x4\2.0.0. Subsequent calls will reuse this data.


>### Flip, crop and resize images

In [ ]:
def func_flip_LR(lowres_img, highres_img):
    '''Flips Images to left and right'''
    
    # outputs random values from a uniform distribution in between 0 to 1
    rn = tf.random.uniform(shape=(), maxval=1)
    
    return tf.cond(rn < 0.5,
                   lambda: (lowres_img, highres_img),
                   lambda: (tf.image.flip_left_right(lowres_img), tf.image.flip_left_right(highres_img),
                            ),
                   )
    

def func_random_crop(lowres_img, highres_img, hr_crop_size=96, scale=4):
    '''Crop images
    low resolution images: 24 x 24
    high resolution images: 96 x 96
    '''
    
    lowres_crop_size = hr_crop_size // scale            # 96 //4 =24
    lowres_img_shape = tf.shape(lowres_img)[:2]         # height, width
    
    lowres_width = tf.random.uniform(
        shape=(), maxval=lowres_img_shape[1] - lowres_crop_size + 1, dtype=tf.int32
        )
    lowres_height = tf.random.uniform(
        shape=(), malval=lowres_img_shape[0] - lowres_crop_size + 1, dtype=tf.int32
        )
    
    highres_width = lowres_width * scale
    highres_height = lowres_height * scale

    lowres_img_cropped = lowres_img[
        lowres_height : lowres_height + lowres_crop_size,
        lowres_width : lowres_width + lowres_crop_size,
    ]  # 24x24
    highres_img_cropped = highres_img[
        highres_height : highres_height + hr_crop_size,
        highres_width : highres_width + hr_crop_size,
    ]  # 96x96

    return lowres_img_cropped, highres_img_cropped